In [3]:
import numpy as np

In [54]:
k = 6
N = 15
num_trials = 2 ** k 

sum(np.random.binomial(num_trials,.25,1000000) >= N)/1000000

0.659713

# Finding a Protein Motif

In [50]:
# getting a protein fasta sequence from uniprot database.
from urllib.request import urlopen

url = "http://www.uniprot.org/uniprot/B5ZC00.fasta"
response = urlopen(url)
fasta = response.read().decode('utf-8', 'ignore')

print(fasta)

>sp|B5ZC00|SYG_UREU1 Glycine--tRNA ligase OS=Ureaplasma urealyticum serovar 10 (strain ATCC 33699 / Western) OX=565575 GN=glyQS PE=3 SV=1
MKNKFKTQEELVNHLKTVGFVFANSEIYNGLANAWDYGPLGVLLKNNLKNLWWKEFVTKQ
KDVVGLDSAIILNPLVWKASGHLDNFSDPLIDCKNCKARYRADKLIESFDENIHIAENSS
NEEFAKVLNDYEISCPTCKQFNWTEIRHFNLMFKTYQGVIEDAKNVVYLRPETAQGIFVN
FKNVQRSMRLHLPFGIAQIGKSFRNEITPGNFIFRTREFEQMEIEFFLKEESAYDIFDKY
LNQIENWLVSACGLSLNNLRKHEHPKEELSHYSKKTIDFEYNFLHGFSELYGIAYRTNYD
LSVHMNLSKKDLTYFDEQTKEKYVPHVIEPSVGVERLLYAILTEATFIEKLENDDERILM
DLKYDLAPYKIAVMPLVNKLKDKAEEIYGKILDLNISATFDNSGSIGKRYRRQDAIGTIY
CLTIDFDSLDDQQDPSFTIRERNSMAQKRIKLSELPLYLNQKAHEDFQRQCQK



In [92]:
pattern = '[A-Z]{10,}'
sequence = ''.join(re.findall(pattern, fasta.replace('\n', '')))
sequence


'MKNKFKTQEELVNHLKTVGFVFANSEIYNGLANAWDYGPLGVLLKNNLKNLWWKEFVTKQKDVVGLDSAIILNPLVWKASGHLDNFSDPLIDCKNCKARYRADKLIESFDENIHIAENSSNEEFAKVLNDYEISCPTCKQFNWTEIRHFNLMFKTYQGVIEDAKNVVYLRPETAQGIFVNFKNVQRSMRLHLPFGIAQIGKSFRNEITPGNFIFRTREFEQMEIEFFLKEESAYDIFDKYLNQIENWLVSACGLSLNNLRKHEHPKEELSHYSKKTIDFEYNFLHGFSELYGIAYRTNYDLSVHMNLSKKDLTYFDEQTKEKYVPHVIEPSVGVERLLYAILTEATFIEKLENDDERILMDLKYDLAPYKIAVMPLVNKLKDKAEEIYGKILDLNISATFDNSGSIGKRYRRQDAIGTIYCLTIDFDSLDDQQDPSFTIRERNSMAQKRIKLSELPLYLNQKAHEDFQRQCQK'

In [2]:
# finding motifs in the sequence.
# N-glycosylation motif is written as N{P}[ST]{P}.

import re
pattern_match = '(?=([N][^P][S|T][^P]))'
matches = re.finditer(pattern_match, 'NNTNNTNNTNSKNSDCCIUUIHSDCNZTG')
b = [match.start() + 1 for match in matches]
b


[1, 4, 7, 26]

In [96]:
# matching
pattern_match = '(?=([N][^P][S|T][^P]))'
matches = re.finditer(pattern_match, sequence)
matches_list = [str(match.start() + 1) for match in matches]
matches_str = ' '.join(matches_list)
print(matches_str)

85 118 142 306 395


In [3]:
# Final Solution

# imports
import requests
import re

# getting urls of all proteins
with open('input.txt', 'r') as f:
    proteins = f.read().split()
    # print(proteins)
protein_fastas = []
for z in range(len(proteins)):
    url = 'http://www.uniprot.org/uniprot/' + proteins[z] + '.fasta'
    r = requests.get(url)

    if r.status_code == 300:
        url = 'https://www.uniprot.org/uniprot/?query=replaces:' + proteins[z] +'&format=fasta'
        txt = requests.get(url).text.split('>')
        fasta = txt[1]
        sequence = ''.join(re.findall('[A-Z]{10,}', fasta.replace('\n', '')))
        protein_fastas.append(sequence)
    else:
        fasta = r.text
        sequence = ''.join(re.findall('[A-Z]{10,}', fasta.replace('\n', '')))
        protein_fastas.append(sequence)

# print(protein_fastas)

# matching motif for each sequence and printing indexes of matches.

for idx, sequence in enumerate(protein_fastas):
    matches = re.finditer('(?=([N][^P][S|T][^P]))', sequence)
    matches_list = [str(match.start() + 1) for match in matches]
    matches_str = ' '.join(matches_list)
    if matches_str != '':
        print(f'{proteins[idx]}\n{matches_str}')

P02974_FMM1_NEIGO
67 68 121
P01880_DTC_HUMAN
225 316 367
Q13VE3
95
B3CNE0
107
Q32LI2
157
P49286
4 130
P00740_FA9_HUMAN
203 213
P07987_GUX2_TRIRE
38 313 334
P01045_KNH2_BOVIN
47 87 168 169 197 204 280
B3ET80
6
A9QYR8
183 210
P09791_PARB_TRYBB
56


In [146]:
url = 'https://www.uniprot.org/uniprot/?query=replaces:P0C5G9&format=fasta'
url2 = 'https://www.uniprot.org/uniprot/P0C5G9.fasta'
import requests

r = requests.get(url2)
if r.status_code == 300:
    url = 'https://www.uniprot.org/uniprot/?query=replaces:P0C5G9&format=fasta'

q = requests.get(url)    
# print(q.status_code)
# print(q.text)
txt = q.text
txt2 = txt.split('>')[1]
print(txt2)




sp|P9WKH5|YIA4_MYCTU Insertion element IS6110 uncharacterized 12.0 kDa protein OS=Mycobacterium tuberculosis (strain ATCC 25618 / H37Rv) OX=83332 GN=Rv0795 PE=1 SV=1
MSGGSSRRYPPELRERAVRMVAEIRGQHDSEWAAISEVARLLGVGCAETVRKWVRQAQVD
AGARPGTTTEESAELKRLRRDNAELRRANAILKTASAFFAAELDRPAR



Top Rosalind Solution


In [4]:
from re import finditer
# from sys import argv
from urllib.request import urlopen

uniprot = "http://www.uniprot.org/uniprot/%s.fasta"

for protein in open('input.txt', 'r').read().strip().splitlines():

    # Fetch the protein's fasta file and get rid of newlines.'
    f = urlopen(uniprot % protein).read().decode('utf-8')
    f = ''.join(f.splitlines()[1:])

    # Find all the positions of the N-glycosylation motif.
    locs = [g.start()+1 for g in finditer(r'(?=N[^P][ST][^P])', f)]

    # Print them out, if any.
    if locs != []:
        print(protein)
        print(' '.join(map(str, locs)))

P02974_FMM1_NEIGO
67 68 121
P01880_DTC_HUMAN
225 316 367
Q13VE3
95
B3CNE0
107
Q32LI2
157
P49286
4 130
P00740_FA9_HUMAN
203 213
P07987_GUX2_TRIRE
38 313 334
P01045_KNH2_BOVIN
47 87 168 169 197 204 280
B3ET80
6
A9QYR8
183 210
P09791_PARB_TRYBB
56


In [13]:
with open('input.txt', 'r') as f:
    # print(f)
    # c = f.read().split()
    print(f.read().strip().splitlines())
    # print(f.read().strip().splitlines())

['P02974_FMM1_NEIGO', 'P01880_DTC_HUMAN', 'Q13VE3', 'B3CNE0', 'Q32LI2', 'P49286', 'P00740_FA9_HUMAN', 'P07987_GUX2_TRIRE', 'P01045_KNH2_BOVIN', 'B3ET80', 'Q3ATP6', 'A9QYR8', 'P09791_PARB_TRYBB']


In [18]:
# new opening/split method discovered
uniprot = "http://www.uniprot.org/uniprot/%s.fasta"
protein = 'P02974_FMM1_NEIGO'
f = urlopen(uniprot % protein).read().decode('utf-8')
print(f'{f}\n')
f = ''.join(f.splitlines()[1:])
print(f'sequence without all the dogshit\n {f}')
print(f[0])

>sp|P02974|FMM1_NEIGO Type IV major pilin protein PilE1 OS=Neisseria gonorrhoeae OX=485 GN=pilE1 PE=1 SV=2
MNTLQKGFTLIELMIVIAIVGILAAVALPAYQDYTARAQVSEAILLAEGQKSAVTEYYLN
HGKWPENNTSAGVASPPSDIKGKYVKEVEVKNGVVTATMLSSGVNNEIKGKKLSLWARRE
NGSVKWFCGQPVTRTDDDTVADAKDGKEIDTKHLPSTCRDNFDAK


sequence without all the dogshit
 MNTLQKGFTLIELMIVIAIVGILAAVALPAYQDYTARAQVSEAILLAEGQKSAVTEYYLNHGKWPENNTSAGVASPPSDIKGKYVKEVEVKNGVVTATMLSSGVNNEIKGKKLSLWARRENGSVKWFCGQPVTRTDDDTVADAKDGKEIDTKHLPSTCRDNFDAK
M
